In [1]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

print('Modules imported.')

Modules imported.


# Import and parse the dataset

In [47]:
train = pd.read_csv('./dataset/train.csv')

### Split pickup and dropoff columns to corresponding date and time columns

In [48]:
train['pickup_date'], train['pickup_time'] = zip(*train['pickup_datetime'].apply(lambda x: x.split(' ')))
train['dropoff_date'], train['dropoff_time'] = zip(*train['dropoff_datetime'].apply(lambda x: x.split(' ')))

In [50]:
pickup_date_year, pickup_date_month, pickup_date_day = zip(*train['pickup_date'].apply(lambda x: x.split('-')))
dropoff_date_year, dropoff_date_month, dropoff_date_day = zip(*train['dropoff_date'].apply(lambda x: x.split('-')))

In [51]:
pickup_date_month = pd.to_numeric(pickup_date_month)
pickup_date_day = pd.to_numeric(pickup_date_day)

dropoff_date_month = pd.to_numeric(dropoff_date_month)
dropoff_date_day = pd.to_numeric(dropoff_date_day)

In [55]:
pickup_days = (pickup_date_month * 30) + pickup_date_day
dropoff_days = (dropoff_date_month * 30) + dropoff_date_day

train['pickup_days'] = pickup_days
train['dropoff_days'] = dropoff_days

In [63]:
pickup_hours = pd.to_numeric(train.pickup_time.str.slice(0, 2))
dropoff_hours = pd.to_numeric(train.dropoff_time.str.slice(0, 2))

train['pickup_hours'] = pickup_hours
train['dropoff_hours'] = dropoff_hours

In [64]:
fwd_flag = pd.get_dummies(train.store_and_fwd_flag).iloc[:,1]
train['fwd_flag'] = fwd_flag

In [65]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_date,pickup_time,dropoff_date,dropoff_time,pickup_days,dropoff_days,fwd_flag,pickup_hours,dropoff_hours
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016-03-14,17:24:55,2016-03-14,17:32:30,104,104,0,17,17
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,2016-06-12,00:43:35,2016-06-12,00:54:38,192,192,0,0,0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,2016-01-19,11:35:24,2016-01-19,12:10:48,49,49,0,11,12
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,2016-04-06,19:32:31,2016-04-06,19:39:40,126,126,0,19,19
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,2016-03-26,13:30:55,2016-03-26,13:38:10,116,116,0,13,13
